In [8]:
import numpy as np
from gurobipy import *
import random
from colormap import rgb2hex
import pandas as pd
import networkx as nx
import numpy as np

In [83]:
import random
import math
import copy


class SimulatedAnnealing:
    def __init__(self, initialSolution, initialTemp, finalTemp, tempReduction, linear_comp, quadratic_comp, coef, iterationPerTemp=100, alpha=0.1, beta=5):
        self.solution = initialSolution
        self.currTemp = initialTemp
        self.finalTemp = finalTemp
        self.iterationPerTemp = iterationPerTemp
        self.alpha = alpha
        self.beta = beta

        if tempReduction == "linear":
            self.decrementRule = self.linearTempReduction
        elif tempReduction == "geometric":
            self.decrementRule = self.geometricTempReduction
        elif tempReduction == "slowDecrease":
            self.decrementRule = self.slowDecreaseTempReduction
        else:
            self.decrementRule = tempReduction
        
        self.qubo = {'linear': linear_comp, 'quadratic': quadratic_comp, 'coefficient': coef}
        
        self.lim = coef
        for obj in self.qubo['linear'].keys():
            self.lim = self.lim + self.qubo['linear'][obj]
        for obj in self.qubo['quadratic'].keys():
            self.lim = self.lim + self.qubo['quadratic'][obj]
        

    def linearTempReduction(self):
        self.currTemp -= self.alpha

    def geometricTempReduction(self):
        self.currTemp *= self.alpha

    def slowDecreaseTempReduction(self):
        self.currTemp = self.currTemp / (1 + self.beta * self.currTemp)

    def isTerminationCriteriaMet(self):
        # can add more termination criteria
        return self.currTemp <= self.finalTemp or len(self.neighborOperator(self.solution)) == 0
    
    def evaluate(self, solution):
        res = self.qubo['coefficient']
        for obj in self.qubo['linear'].keys():
            res = res + self.qubo['linear'][obj]*solution[obj]
        for obj in self.qubo['quadratic'].keys():
            res = res + self.qubo['quadratic'][obj]*solution[obj[0]]*solution[obj[1]]
        return res/self.lim
    
    def neighborOperator(self, solution):
        neighbors = []
        
        for obj in self.qubo['linear'].keys():
            new_neighbor = copy.deepcopy(solution)
            new_neighbor[obj] = 1 - solution[obj]
            neighbors.append(new_neighbor)
        
        return neighbors
            

    def run(self):
        while not self.isTerminationCriteriaMet():
            # iterate that number of times
            print('TEMP: ', self.currTemp)
            for i in range(self.iterationPerTemp):
                # get all of the neighbors
                neighbors = self.neighborOperator(self.solution)
                # pick a random neighbor
                newSolution = random.choice(neighbors)
                # get the cost between the two solutions
                cost = self.evaluate(self.solution) - self.evaluate(newSolution)
                # if the new solution is better, accept it
                if cost >= 0:
                    self.solution = newSolution
                # if the new solution is not better, accept it with a probability of e^(-cost/temp)
                else:
                    #print(-cost, self.currTemp, -cost / self.currTemp)
                    if random.uniform(0, 1) < math.exp(-cost / self.currTemp):
                        self.solution = newSolution
            # decrement the temperature
            self.decrementRule()
            print(self.evaluate(self.solution)*self.lim)
        #return self.evaluate(self.solution)*self.lim
        

In [9]:
from scipy import optimize

def evaluate_function(sol, *params):
    
    qubo, mapping, scale = params
    #print(mapping)
    res = qubo['coefficient']
    for obj in qubo['linear'].keys():
        res = res + qubo['linear'][obj]*sol[mapping[obj]]
    for obj in qubo['quadratic'].keys():
        res = res + qubo['quadratic'][obj]*sol[mapping[obj[0]]]*sol[mapping[obj[1]]]
    return res/scale

In [9]:
target = ['C00074']
qubo_comp, coef = qubo_initialize(list_enzymes, list_reactions, list_compounds, target, G)
linear_comp, quadratic_comp = qubo_splitting(qubo_comp)
qubo = {'linear': linear_comp, 'quadratic': quadratic_comp, 'coefficient': coef}

In [10]:
init_solution = []
mapping = {}
index = 0
for obj in linear_comp.keys():
    init_solution.append(random.choice([0, 1]))
    mapping[obj] = index
    index = index + 1
    
init_solution = np.array(init_solution)

scale = coef
for obj in qubo['linear'].keys():
    scale = scale + qubo['linear'][obj]
    
params = [qubo, mapping, scale]
lw = [0] * len(linear_comp.keys())
up = [1] * len(linear_comp.keys())
res = optimize.dual_annealing(evaluate_function, list(zip(lw, up)), params, 100)

In [19]:
res

 message: ['Maximum number of iteration reached']
 success: True
  status: 0
     fun: 0.01683945541752316
       x: [ 0.000e+00  1.000e+00 ...  1.000e+00  0.000e+00]
     nit: 100
    nfev: 50481
    njev: 80
    nhev: 0

In [10]:
def unpacking(res_packet, list_compounds, target, mapping):
    remain, remain_list, damage, damage_list = 0, [], 0, []
    checking = {}
    for i in range(len(res_packet.x)):
        if res_packet.x[i] > 0.5:
            checking[mapping[i]] = 1
        else:
            checking[mapping[i]] = 0
            
    for comp in list_compounds:
        if comp in target:
            if checking[comp] == 0:
                remain = remain + 1
                remain_list.append(comp)
        else:
            if checking[comp] == 1:
                damage = damage + 1
                damage_list.append(comp)
    
    return remain, remain_list, damage, damage_list
            

In [15]:
#pathway_list = ['Purine', 'Carbon metabolism', 'Nucleotide metabolism', 'Biosynthesis of amino acids', 'Biosynthesis of cofactors']
pathway_list = ['Glycerolipid', 'Citrate_cycle', 'Pentose_phosphate', 'Pyruvate', 'Galactose']
pathway_list = ['Glycerolipid', 'Citrate_cycle', 'Pentose_phosphate', 'Glycine_serine_threonine', 'Galactose']
#pathway_list = ['Citrate_cycle']


#species_list = ['hsa', 'mmu']
species_list = ['eco']
df = pd.DataFrame(columns=['Species', 'Pathway', 'Target', 'Remain', 'Remain_Target', 'SA_damage', 'QC_damage'])
data_type = 'small'

nums_invalid = 0
nums_total = 0

for species_name  in species_list:
    for pathway_name in pathway_list:
        #print(pathway_name)
        list_enzymes = []
        list_reactions = []
        list_compounds = []
        nodes = []
        edges = []
        mode = 0
        num_edges = 0
        with open(data_type+'_data/'+species_name+'_filtering/'+pathway_name+'.input') as f:
            while mode <= 3:
                if mode < 3:
                    num_nodes = int(f.readline())
                      #print(num_nodes)
                    for i in range(0, num_nodes):
                        element = f.readline()
                        #print(element)
                        nodes.append(element[0:-1])
                        if mode == 0:
                            list_enzymes.append(element[0:-1])
                        elif mode == 1:
                            list_reactions.append(element[0:-1])
                        else:
                            list_compounds.append(element[0:-1])
                    mode = mode + 1
                else:
                    num_edges = int(f.readline())
                    for i in range(0, num_edges):
                        line = f.readline()
                        element = line[0:-1].split(' ')
                        edges.append(element)
                    mode = mode + 1
            
        G = nx.DiGraph()
        G.add_edges_from(edges)
        
        list_removal = []
        for comp in G.nodes:
            list_predecessor = [i for i in G.predecessors(comp)]
            if comp in list_compounds and len(list_predecessor) == 0:
                list_removal.append(comp)
                list_compounds.remove(comp)
        G.remove_nodes_from(list_removal)

        is_revertable = {}
        for comp in list_reactions:
            list_predecessor = [i for i in G.predecessors(comp)]
            list_successor = [i for i in G.neighbors(comp)]
            check = False
            for comp_2 in list_predecessor:
                if comp_2 in list_successor:
                    #print(comp_2, comp)
                    G.remove_edge(comp_2, comp)
                    check = True
                    #break
            is_revertable[comp] = check

        idx = 0
        for node in G.nodes:
            G.nodes[node]['index'] = idx
            idx = idx + 1 
        
        #qubo_com, coef = 
        testcase = []
        #print(len(edges), len(G.edges), len(list_enzymes), len(list_reactions), len(list_compounds))
        #break
        with open(data_type+'_output/'+species_name+'/'+pathway_name+'.out') as f:
            while True:
                record = {}
                line = f.readline()
                if line == '':
                    break
                line = f.readline()
                record['Target_size'] = int(line)

                line = f.readline()
                target = []
                for i in range(0, record['Target_size']):
                    line = f.readline()
                    target.append(line[0:-1])
                record['Target'] = target

                line = f.readline()
                line = f.readline()
                record['Exact_damage'] = int(line)

                #if pathway_name != 'Purine':
                line = f.readline()
                line = f.readline()
                record['Exact_running_time'] = float(line)

                line = f.readline()
                line = f.readline()
                record['QC_damage'] = float(line)

                line = f.readline()
                line = f.readline()
                record['QC_QPU_access_time'] = float(line)

                line = f.readline()
                line = f.readline()
                record['QC_running_time'] = float(line)
                
                testcase.append(record)
        #print(testcase)
        i = 0
        for record in testcase:
            #if i!=2:
            #    i = i+1
            #    continue
            #i = i + 1
            #print(list_compounds)
            #print(record['Target'])
            #print(record['DI_damage'])
            target = record['Target']
            qubo_comp, coef = qubo_initialize(list_enzymes, list_reactions, list_compounds, target, G)
            linear_comp, quadratic_comp = qubo_splitting(qubo_comp)
            qubo = {'linear': linear_comp, 'quadratic': quadratic_comp, 'coefficient': coef}
            init_solution = []
            mapping = {}
            index = 0
            for obj in linear_comp.keys():
                init_solution.append(random.choice([0, 1]))
                mapping[obj] = index
                mapping[index] = obj
                index = index + 1

            init_solution = np.array(init_solution)

            scale = coef
            for obj in qubo['linear'].keys():
                scale = scale + qubo['linear'][obj]

            params = [qubo, mapping, scale]
            lw = [0] * len(linear_comp.keys())
            up = [1] * len(linear_comp.keys())
            res_packet = optimize.dual_annealing(evaluate_function, list(zip(lw, up)), params, 100)
            remain, remain_list, damage, damage_list = unpacking(res_packet, list_compounds, target, mapping)
            print(target, remain, remain_list, damage, damage_list)
            rec = {'Species': species_name, 'Pathway': pathway_name, 'Target':target, 'Remain':remain, 'Remain_Target':remain_list, 'SA_damage':damage, 'QC_damage':record['QC_damage']}
            df = df.append(rec, ignore_index=True)
            #break
            if i == 0:
                f = open(data_type+"_output/"+species_name+"/SA_Res/"+pathway_name+".out", "w")
                i = i + 1
            else:
                f = open(data_type+"_output/"+species_name+"/SA_Res/"+pathway_name+".out", "a")
            f.write('Target_size\n')
            f.write(str(record['Target_size'])+'\n')
            f.write('Target\n')
            for enzy in record['Target']:
                f.write(enzy+'\n')
            f.write('Exact_damage\n')
            f.write(str(record['Exact_damage'])+'\n')
            f.write('Exact_running_time\n')
            f.write(str(record['Exact_running_time'])+'\n')
            f.write('QC_damage\n')
            f.write(str(record['QC_damage'])+'\n')
            f.write('QC_QPU_access_time\n')
            f.write(str(record['QC_QPU_access_time'])+'\n')
            f.write('QC_running_time\n')
            f.write(str(record['QC_running_time'])+'\n')
            f.write('SA_remain\n')
            f.write(str(remain)+'\n')
            f.write('SA_damage\n')
            f.write(str(damage)+'\n')
            f.close()

['C00416', 'C02133'] 1 ['C00416'] 4 ['C03692', 'C00577', 'C00681', 'C00093']


/tmp/ipykernel_1380361/3420422285.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(rec, ignore_index=True)


['C02133', 'C00681'] 0 [] 6 ['C00416', 'C03692', 'C00577', 'C00111', 'C00093', 'C00116']


/tmp/ipykernel_1380361/3420422285.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(rec, ignore_index=True)


['C00681', 'C00116'] 1 ['C00681'] 6 ['C00416', 'C00631', 'C02133', 'C03692', 'C00577', 'C00093']


...

/tmp/ipykernel_1380361/3420422285.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(rec, ignore_index=True)


['C00446', 'C00137', 'C03733', 'C01216', 'C00029', 'C03785', 'C00103', 'C01097', 'C00031', 'C00492', 'C05401', 'C00159', 'C00111', 'C00668', 'C01286', 'C00095', 'C00794', 'C00052', 'C00984', 'C06311', 'C00267', 'C00118', 'C00116', 'C06376', 'C05400', 'C05402', 'C00124'] 12 ['C00446', 'C00029', 'C00103', 'C01097', 'C00031', 'C00492', 'C05401', 'C00668', 'C00267', 'C00116', 'C05402', 'C00124'] 2 ['C05399', 'C01235']


/tmp/ipykernel_1380361/3420422285.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(rec, ignore_index=True)


['C00446', 'C00137', 'C03733', 'C00089', 'C00029', 'C03785', 'C00103', 'C01097', 'C00031', 'C00492', 'C05401', 'C00159', 'C00111', 'C00668', 'C01286', 'C06377', 'C00095', 'C00794', 'C00052', 'C00984', 'C06311', 'C00118', 'C00116', 'C06376', 'C01235', 'C05400', 'C00124'] 2 ['C00492', 'C00124'] 3 ['C01216', 'C00267', 'C05399']


/tmp/ipykernel_1380361/3420422285.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(rec, ignore_index=True)


In [32]:
df

,Species,Pathway,Target,Remain,Remain_Target,SA_damage,QC_damage
0,hsa,Glycerolipid,"[C00258, C00681]",1,[C00258],5,1.0
1,hsa,Glycerolipid,"[C00116, C00681]",0,[],8,1.0
2,hsa,Glycerolipid,"[C00416, C00422]",1,[C00416],6,0.0
3,hsa,Glycerolipid,"[C00641, C00258]",2,"[C00641, C00258]",4,4.0
4,hsa,Glycerolipid,"[C00093, C00631]",0,[],8,0.0
5,hsa,Glycerolipid,"[C00641, C01885, C00111, C00162, C03692, C0063...",1,[C03692],5,0.0
6,hsa,Glycerolipid,"[C00258, C00416, C00681, C00162, C03692, C0009...",4,"[C00258, C00416, C00162, C00422]",3,3.0
7,hsa,Glycerolipid,"[C00641, C01885, C00116, C00111, C00162, C0369...",2,"[C00111, C03692]",5,3.0
8,hsa,Glycerolipid,"[C00641, C01885, C00116, C00111, C00258, C0369...",2,"[C00641, C00422]",3,6.0
9,hsa,Glycerolipid,"[C00116, C00258, C00416, C00577, C00681, C0009...",2,"[C00681, C00093]",4,0.0


In [11]:
def qubo_initialize(list_enzymes, list_reactions, list_compounds, target, G):
  C = target
  qubo_comp = {}
  A1 = 1
  A2 = 300
  A3 = 250
  A4 = 250
  coefficient = 0
  for comp in list_compounds:
    if comp not in C:
      qubo_comp[comp] = A1
    else:
      qubo_comp[comp] = -A2
      coefficient = coefficient + A2

  for comp_i in list_compounds:
    list_predecessor = [i for i in G.predecessors(comp_i)]

    for comp_j in list_predecessor:
      temp_comp = extract_quadratic_func({comp_i: 1, comp_j: -1})
      qubo_comp = combine_comp(qubo_comp, temp_comp, A3)
    
    for comp_j in list_predecessor:
      temp_comp = extract_quadratic_func({comp_i: 1, comp_j: -1, 1:1})
      qubo_comp = combine_comp(qubo_comp, temp_comp, A3)
      #coefficient = coefficient + A3
    
    input_comp = {comp_i: -1, 1:1}
    for comp_j in list_predecessor:
      input_comp[comp_j] = 1
    
    for i in range(0, len(list_predecessor) + 1):
      input_comp['W_'+str(comp_i)+'_'+str(i)] = -i
    
    temp_comp = extract_quadratic_func(input_comp)
    qubo_comp = combine_comp(qubo_comp, temp_comp, A3)
    coefficient = coefficient + A3

    input_comp = {1:1}
    for i in range(0, len(list_predecessor) + 1):
      input_comp['W_'+str(comp_i)+'_'+str(i)] = -1
    
    temp_comp = extract_quadratic_func(input_comp)
    qubo_comp = combine_comp(qubo_comp, temp_comp, A3)
    coefficient = coefficient + A3

  for comp_i in list_reactions:
    list_predecessor = [i for i in G.predecessors(comp_i)]
    removal_list = []
    if is_revertable[comp_i]:
      for comp_j in list_predecessor:
        if comp_j in list_compounds:
          removal_list.append(comp_j)
    for comp_j in removal_list:
      list_predecessor.remove(comp_j)
    for comp_j in list_predecessor:
      temp_comp = extract_quadratic_func({comp_i: 1, comp_j: -1})
      qubo_comp = combine_comp(qubo_comp, temp_comp, A4)
    
    for comp_j in list_predecessor:
      temp_comp = extract_quadratic_func({comp_i: 1, comp_j: -1, 1:-1})
      qubo_comp = combine_comp(qubo_comp, temp_comp, A4)
      #print(comp_i, temp_comp)
      #coefficient = coefficient - A4
    
    input_comp = {comp_i: 1}
    for comp_j in list_predecessor:
      input_comp[comp_j] = -1
    
    for i in range(0, len(list_predecessor) + 1):
      input_comp['T_'+str(comp_i)+'_'+str(i)] = i
    
    temp_comp = extract_quadratic_func(input_comp)
    qubo_comp = combine_comp(qubo_comp, temp_comp, A4)

    input_comp = {1:1}
    for i in range(0, len(list_predecessor) + 1):
      input_comp['T_'+str(comp_i)+'_'+str(i)] = -1
    
    temp_comp = extract_quadratic_func(input_comp)
    qubo_comp = combine_comp(qubo_comp, temp_comp, A4)
    coefficient = coefficient + A4
  return qubo_comp, coefficient

In [12]:
def bigger(k1, k2):
  if type(k1) is type(k2):
    return k1 > k2
  else:
    if type(k1) is int:
      return False
    else:
      return True

def extract_quadratic_func(input_comp):
  temp_comp = {}
  keys = input_comp.keys()
  for k in keys:
    if type(k) is not int:
      temp_comp[k] = input_comp[k]*input_comp[k]
  for k1 in keys:
    for k2 in keys:
      if (k1 != k2) and (bigger(k1, k2)):
        if type(k1) is int:
          if type(k2) is not int:
            temp_comp[k2] = temp_comp[k2] + 2*input_comp[k1]*input_comp[k2]
        else:
          if type(k2) is int:
            temp_comp[k1] = temp_comp[k1] + 2*input_comp[k2]*input_comp[k1]
          else:
            if (k1, k2) not in temp_comp.keys():
              temp_comp[(k1, k2)] = 2*input_comp[k1]*input_comp[k2]
            else:
               temp_comp[(k1, k2)] = temp_comp[(k1, k2)] + 2*input_comp[k1]*input_comp[k2]
  return temp_comp

def combine_comp(qubo_comp, temp_comp, ins):
  keys = temp_comp.keys()
  for k in keys:
    if k not in qubo_comp:
      qubo_comp[k] = temp_comp[k] * ins
    else:
      qubo_comp[k] = qubo_comp[k] + temp_comp[k] * ins
  return qubo_comp

In [13]:
def qubo_splitting(qubo_comp):
  linear_comp = {}
  quadratic_comp = {}
  for k in qubo_comp.keys():
    if type(k) is str:
      linear_comp[k] = qubo_comp[k]
    else:
      quadratic_comp[k] = qubo_comp[k]
  return linear_comp, quadratic_comp